In [2]:
import numpy as np
import pylab as pl
import scipy as sp
import json
import csv
import seaborn as sns
import tensorflow as tf
from tensorflow import keras

# Input your own file path to https://github.com/vanferdi/resource-rational-prediction/blob/main/Data/experiment.csv
file_path = 'experiment.csv'

Inputs = []
Guesses = []
ParticipantIDs = []
Input_Types = []
Hidden_states = []
Accuracy = []
Bonus_round = []

with open(file_path) as csvfile:
		spamreader = csv.reader(csvfile)
		for row in spamreader:
			input_ = row[7]
			guess_ = row[6]
			ID = row[0]
			hmm_states = row[13]
			accuracy = row[22]
			bonus_round = row[29]
			Inputs.append(input_)
			Guesses.append(guess_)
			ParticipantIDs.append(ID)
			Input_Types.append(row[15])
			Hidden_states.append(hmm_states)
			Accuracy.append(accuracy)
			Bonus_round.append(bonus_round)
		Inputs.pop(0)
		Guesses.pop(0)
		ParticipantIDs.pop(0)
		Input_Types.pop(0)
		Hidden_states.pop(0)
		Accuracy.pop(0)
		Bonus_round.pop(0)

In [ ]:
Input_Types[1]

'NoisyPeriodic'

In [ ]:
len(Inputs[1])

445

In [3]:
def rate(hidden_states,guesses):
	# get statistics
	# this only works when things are well-sampled
	joint_xy = {}
	marginal_x = {}
	marginal_y = {}
	for i in range(len(hidden_states)):
		xy = str(hidden_states[i])+str(guesses[i])
		x = str(hidden_states[i])
		y = str(guesses[i])
		if xy in joint_xy:
			joint_xy[xy] = joint_xy[xy]+1
		else:
			joint_xy[xy] = 1
		if x in marginal_x:
			marginal_x[x] = marginal_x[x]+1
		else:
			marginal_x[x] = 1
		if y in marginal_y:
			marginal_y[y] = marginal_y[y]+1
		else:
			marginal_y[y] = 1
	# hack to add appropriate pseudocounts
	all_cs = np.unique(hidden_states)
	all_output = [0,1]
	if len(marginal_y)<2:
		marginal_y[str(2)] = 0
	for cs in all_cs:
		for output in all_output:
			xy = str(cs)+str(output)
			if xy in joint_xy:
				pass
			else:
				joint_xy[xy] = 0
	# find tot
	tot_xy = np.sum(list(joint_xy.values()))
	tot_x = np.sum(list(marginal_x.values()))
	tot_y = np.sum(list(marginal_y.values()))
	# get the entropies
	p_xy = np.asarray(list(joint_xy.values()))/tot_xy
	H_xy = -np.nansum(p_xy*np.log2(p_xy))
	p_x = np.asarray(list(marginal_x.values()))/tot_x
	H_x = -np.nansum(p_x*np.log2(p_x))
	p_y = np.asarray(list(marginal_y.values()))/tot_y
	H_y = -np.nansum(p_y*np.log2(p_y))
	# get the mutual information
	I_xy = H_x+H_y-H_xy
	return I_xy

In [4]:
def ngram_learner(n,data_input,hidden_states):
	# first rate and accuracy is with no probability matching
	# second rate and accuracy is with probability matching
	data_input1 = []
	hidden_states1 = []
	guesses1 = []
	guesses2 = []
	for i in range(n+1,len(data_input)):
		# figure out what the probabilities are of guessing
		# last n symbols
		foo = data_input[i-n-1:i-1]
		foo0 = foo+'0'
		foo1 = foo+'1'
		p0 = data_input[:i].count(foo0)+1
		p1 = data_input[:i].count(foo1)+1
		guess1 = np.argmax([p0,p1])
		guess2 = np.random.choice(2,size=1,p=[p0/(p0+p1),p1/(p0+p1)])
		guesses1.append(guess1)
		guesses2.append(guess2)
		hidden_states1.append(hidden_states[i-1])
		data_input1.append(data_input[i-1])
	# get rates and accuracies
	M = len(guesses1); guesses1 = guesses1[int(M/2):M]
	guesses2 = guesses2[int(M/2):M]
	data_input1 = data_input1[int(M/2):M]; hidden_states1 = hidden_states1[int(M/2):M]
	Rate1 = rate(hidden_states1,guesses1)
	Rate2 = rate(hidden_states1,guesses2)
	Rates = [Rate1,Rate2]
	Acc1 = np.mean([int(data_input1[j])==guesses1[j] for j in range(int(M/2))])
	Acc2 = np.mean([int(data_input1[j])==guesses2[j] for j in range(int(M/2))]); Accuracies = [Acc1,Acc2]
	return Rates, Accuracies

In [7]:
print(Input_Types[0])
print(bad_learners(Inputs[0]))
for i in range(10):
  print(ngram_learner(i,Inputs[0],Hidden_states[0]))

NoisyPeriodic
([0, 0], [0.5, np.float64(0.56)])
([np.float64(0.0), np.float64(0.001171354295115723)], [np.float64(0.5678391959798995), np.float64(0.46733668341708545)])
([np.float64(0.6930925963145924), np.float64(0.5138594483586991)], [np.float64(0.8592964824120602), np.float64(0.8241206030150754)])
([np.float64(0.9286161267297095), np.float64(0.3698899111107872)], [np.float64(0.9393939393939394), np.float64(0.7929292929292929)])
([np.float64(0.9284189173229802), np.float64(0.5085708485336398)], [np.float64(0.9191919191919192), np.float64(0.8535353535353535)])
([np.float64(1.0), np.float64(0.5704901020119046)], [np.float64(0.9289340101522843), np.float64(0.8629441624365483)])
([np.float64(0.9999814128006821), np.float64(0.4463278339194101)], [np.float64(0.934010152284264), np.float64(0.8223350253807107)])
([np.float64(0.9999814128006821), np.float64(0.41597076899278074)], [np.float64(0.9336734693877551), np.float64(0.8010204081632653)])


/tmp/ipython-input-2166640528.py:41: RuntimeWarning: divide by zero encountered in log2
  H_xy = -np.nansum(p_xy*np.log2(p_xy))
/tmp/ipython-input-2166640528.py:41: RuntimeWarning: invalid value encountered in multiply
  H_xy = -np.nansum(p_xy*np.log2(p_xy))
/tmp/ipython-input-2166640528.py:45: RuntimeWarning: divide by zero encountered in log2
  H_y = -np.nansum(p_y*np.log2(p_y))
/tmp/ipython-input-2166640528.py:45: RuntimeWarning: invalid value encountered in multiply
  H_y = -np.nansum(p_y*np.log2(p_y))


([np.float64(1.0), np.float64(0.34412977252891497)], [np.float64(0.9336734693877551), np.float64(0.7959183673469388)])
([np.float64(0.9588533639561687), np.float64(0.32679869513985227)], [np.float64(0.9384615384615385), np.float64(0.7897435897435897)])
([np.float64(0.9586233851340582), np.float64(0.1957756872930636)], [np.float64(0.9384615384615385), np.float64(0.7333333333333333)])


In [8]:
print(Input_Types[1])
print(bad_learners(Inputs[1]))
for i in range(10):
  print(ngram_learner(i,Inputs[1],Hidden_states[1]))

Clumpy
([0, 0], [0.5, np.float64(0.6140888208269525)])
([np.float64(0.0), np.float64(0.006863705450984359)], [np.float64(0.6165644171779141), np.float64(0.5030674846625767)])
([np.float64(0.9583015836671662), np.float64(0.001290502461923193)], [np.float64(0.6184615384615385), np.float64(0.5015384615384615)])
([np.float64(0.7030778604454733), np.float64(0.3867524842481873)], [np.float64(0.8092307692307692), np.float64(0.7323076923076923)])
([np.float64(0.38164381118861623), np.float64(0.30636367649534213)], [np.float64(0.6851851851851852), np.float64(0.7222222222222222)])


/tmp/ipython-input-2166640528.py:41: RuntimeWarning: divide by zero encountered in log2
  H_xy = -np.nansum(p_xy*np.log2(p_xy))
/tmp/ipython-input-2166640528.py:41: RuntimeWarning: invalid value encountered in multiply
  H_xy = -np.nansum(p_xy*np.log2(p_xy))
/tmp/ipython-input-2166640528.py:45: RuntimeWarning: divide by zero encountered in log2
  H_y = -np.nansum(p_y*np.log2(p_y))
/tmp/ipython-input-2166640528.py:45: RuntimeWarning: invalid value encountered in multiply
  H_y = -np.nansum(p_y*np.log2(p_y))


([np.float64(0.3928748672364626), np.float64(0.32508958775591257)], [np.float64(0.7160493827160493), np.float64(0.7345679012345679)])
([np.float64(0.42322104748374123), np.float64(0.2767180793195587)], [np.float64(0.739938080495356), np.float64(0.7027863777089783)])
([np.float64(0.43563357427977767), np.float64(0.2254196678973419)], [np.float64(0.7739938080495357), np.float64(0.6842105263157895)])
([np.float64(0.4724886632217), np.float64(0.27011791840718224)], [np.float64(0.7950310559006211), np.float64(0.7049689440993789)])
([np.float64(0.48385048497789573), np.float64(0.16934510104148837)], [np.float64(0.8322981366459627), np.float64(0.7018633540372671)])
([np.float64(0.45741426460156775), np.float64(0.21440449703042042)], [np.float64(0.8255451713395638), np.float64(0.7320872274143302)])


In [11]:
print(Input_Types[3])
print(bad_learners(Inputs[3]))
for i in range(10):
  print(ngram_learner(i,Inputs[3],Hidden_states[3]))

EvenProcess
([0, 0], [0.5, np.float64(0.5381062355658199)])
([np.float64(0.0), np.float64(3.399370265055879e-06)], [np.float64(0.5462962962962963), np.float64(0.49537037037037035)])
([np.float64(0.3187791725115614), np.float64(0.007911581021301961)], [np.float64(0.6976744186046512), np.float64(0.5767441860465117)])
([np.float64(0.08592381874622945), np.float64(0.06360649611850833)], [np.float64(0.6511627906976745), np.float64(0.5767441860465117)])
([np.float64(0.2140230858094503), np.float64(0.09702967881529534)], [np.float64(0.6261682242990654), np.float64(0.6355140186915887)])
([np.float64(0.2237295050598307), np.float64(0.1311389135735881)], [np.float64(0.6635514018691588), np.float64(0.6401869158878505)])
([np.float64(0.3812150748820715), np.float64(0.18706048930513774)], [np.float64(0.7276995305164319), np.float64(0.6431924882629108)])


/tmp/ipython-input-2166640528.py:41: RuntimeWarning: divide by zero encountered in log2
  H_xy = -np.nansum(p_xy*np.log2(p_xy))
/tmp/ipython-input-2166640528.py:41: RuntimeWarning: invalid value encountered in multiply
  H_xy = -np.nansum(p_xy*np.log2(p_xy))
/tmp/ipython-input-2166640528.py:45: RuntimeWarning: divide by zero encountered in log2
  H_y = -np.nansum(p_y*np.log2(p_y))
/tmp/ipython-input-2166640528.py:45: RuntimeWarning: invalid value encountered in multiply
  H_y = -np.nansum(p_y*np.log2(p_y))


([np.float64(0.35782965580958814), np.float64(0.12865769935130755)], [np.float64(0.7370892018779343), np.float64(0.6431924882629108)])
([np.float64(0.3941190697293615), np.float64(0.08868599944041211)], [np.float64(0.75), np.float64(0.6179245283018868)])
([np.float64(0.43976514570880054), np.float64(0.12500043697444263)], [np.float64(0.8254716981132075), np.float64(0.6792452830188679)])
([np.float64(0.4616419095283124), np.float64(0.08784717857093272)], [np.float64(0.8436018957345972), np.float64(0.6445497630331753)])


In [ ]:
def LSTM_learner(data_input,hidden_states):
	# first rate and accuracy is with no probability matching
	# second rate and accuracy is with probability matching
	timesteps = 5
	data_input1 = []
	hidden_states1 = []
	guesses_1 = [] # with no probability matching
	guesses_2 = [] # with probability matching
	tf.keras.backend.clear_session()
	# train an LSTM to predict in MSE sense
	batch_x = []; batch_y = []
	for i in range(timesteps,len(data_input)):
		batch_x.append([int(data_input[j]) for j in range(i-timesteps,i)])
		batch_y.append([int(data_input[i])])
	batch_x = np.asarray(batch_x)
	batch_y = np.asarray(batch_y)
	batch_x = batch_x.reshape((len(batch_x),timesteps,1))
	batch_y = batch_y.reshape((len(batch_y),1))
	batch_x = tf.convert_to_tensor(batch_x)
	batch_y = tf.convert_to_tensor(batch_y)
	#
	X = tf.keras.Input(name='X', shape=[timesteps, 1], dtype=tf.dtypes.float32)
	#
	lstm_output = tf.keras.layers.LSTM(50)(X)
	prediction = tf.keras.layers.Dense(1,activation='sigmoid')(lstm_output)
	opt = tf.keras.optimizers.Adam(learning_rate=1e-5)
	model = keras.Model(inputs=X, outputs=prediction)
	model.compile(optimizer=opt, loss='binary_crossentropy')
	model.fit(x=batch_x, y=batch_y, epochs=100, verbose=0)
	for i in range(timesteps,len(data_input)):
		batch_x = [int(data_input[j]) for j in range(i-timesteps,i)]
		batch_y = [int(data_input[i])]
		batch_x = np.asarray(batch_x)
		batch_y = np.asarray(batch_y)
		batch_x = batch_x.reshape((1,timesteps,1))
		batch_y = batch_y.reshape((len(batch_y),1))
		batch_x = tf.convert_to_tensor(batch_x)
		batch_y = tf.convert_to_tensor(batch_y)
		xpreds = model.predict(batch_x)
		data_input1.append(int(data_input[i])); #print(batch_y.numpy()[-1])
		hidden_states1.append(hidden_states[i])
		guesses_1.append(np.round(xpreds[0][0]))
		p = np.array([1 - float(xpreds[0][0]), float(xpreds[0][0])], dtype=np.float64)
		guesses_2.append(np.random.choice(a=2,size=1,p=p/np.sum(p)))
	# get rates and accuracies
	M = len(guesses_1); guesses_1 = guesses_1[int(M/2):M]
	guesses_2 = guesses_2[int(M/2):M]
	data_input1 = data_input1[int(M/2):M]; hidden_states1 = hidden_states1[int(M/2):M]
	Rate1 = rate(hidden_states1,guesses_1)
	Rate2 = rate(hidden_states1,guesses_2)
	Rates = [Rate1,Rate2]
	Acc1 = np.mean([np.abs(data_input1[j]-guesses_1[j])<1e-5 for j in range(len(guesses_1))])
	Acc2 = np.mean([np.abs(data_input1[j]-guesses_2[j])<1e-5 for j in range(len(guesses_2))])
	Accuracies = [Acc1,Acc2]
	return Rates, Accuracies

In [6]:
def bad_learners(data_input):
	foo = [int(i) for i in data_input]
	freq1 = np.mean(foo); freq0 = 1-freq1 # best guess
	Rate1 = 0
	Rate2 = 0
	Rates = [Rate1,Rate2]
	Acc1 = 0.5
	Acc2 = np.max([freq0,freq1])
	Accuracies = [Acc1,Acc2]
	return Rates, Accuracies

In [ ]:
r, a = LSTM_learner(Inputs[0],Hidden_states[0])
print(r)
print(a)

1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 229ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 47ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 55ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 50ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 50ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 50ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 50ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 61ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 54ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 52ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 58ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 69ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 55ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 53ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 35ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 39ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 34ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 33ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 35ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 34ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 34ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 34ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 36ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 50ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 36ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 39ms/step
1/1 ━━━━━━━

/tmp/ipython-input-2166640528.py:41: RuntimeWarning: divide by zero encountered in log2
  H_xy = -np.nansum(p_xy*np.log2(p_xy))
/tmp/ipython-input-2166640528.py:41: RuntimeWarning: invalid value encountered in multiply
  H_xy = -np.nansum(p_xy*np.log2(p_xy))
/tmp/ipython-input-2166640528.py:45: RuntimeWarning: divide by zero encountered in log2
  H_y = -np.nansum(p_y*np.log2(p_y))
/tmp/ipython-input-2166640528.py:45: RuntimeWarning: invalid value encountered in multiply
  H_y = -np.nansum(p_y*np.log2(p_y))


In [ ]:
print(len(Inputs[0])); print(len(Hidden_states[0]))

653
653


In [ ]:
r, a = LSTM_learner(Inputs[1],Hidden_states[1])
print(r)
print(a)

1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 170ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 36ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 47ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 38ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 37ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 36ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 35ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 34ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 34ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 34ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 38ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 34ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 34ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 38ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 35ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 49ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 37ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 35ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 38ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 36ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 35ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 37ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 36ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 37ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 35ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 37ms/step
1/1 ━━━━━━━

/tmp/ipython-input-2166640528.py:41: RuntimeWarning: divide by zero encountered in log2
  H_xy = -np.nansum(p_xy*np.log2(p_xy))
/tmp/ipython-input-2166640528.py:41: RuntimeWarning: invalid value encountered in multiply
  H_xy = -np.nansum(p_xy*np.log2(p_xy))


In [ ]:
r, a = LSTM_learner(Inputs[2],Hidden_states[2])
print(r)
print(a)

1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 163ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 37ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 35ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 39ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 34ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 37ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 34ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 33ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 36ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 35ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 35ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 35ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 34ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 38ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 35ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 37ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 37ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 58ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 34ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 34ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 36ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 33ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 34ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 34ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 35ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 37ms/step
1/1 ━━━━━━━

/tmp/ipython-input-2166640528.py:41: RuntimeWarning: divide by zero encountered in log2
  H_xy = -np.nansum(p_xy*np.log2(p_xy))
/tmp/ipython-input-2166640528.py:41: RuntimeWarning: invalid value encountered in multiply
  H_xy = -np.nansum(p_xy*np.log2(p_xy))
/tmp/ipython-input-2166640528.py:45: RuntimeWarning: divide by zero encountered in log2
  H_y = -np.nansum(p_y*np.log2(p_y))
/tmp/ipython-input-2166640528.py:45: RuntimeWarning: invalid value encountered in multiply
  H_y = -np.nansum(p_y*np.log2(p_y))
